# 모델 저장하고 불러오기
---
모델을 저장하고 불러오는 것은 매우 중요한 문제이다. 신경망을 학습시킬때 원큐에 학습이 될거란 보장이 없지 않은가?

그럼 어떻게 저장할까?

In [ ]:
import torch
import torchvision.models as models

### 1. 모델 가중치 저장하고 불러오기
---
> PyTorch 모델은 학습한 매개변수를 state_dict라고 불리는 내부 상태 사전(internal state dictionary)에 저장합니다.  
> 이 상태 값들은 torch.save 메소드를 사용하여 저장(persist)할 수 있습니다:

1. 내부 상태 사전(state_dict)가 뭔데?

    - 그냥 딕셔너리이다. 

    - 단지 각 계층을 매개변수 Tensor로 매핑해서 학습 가능한 매개변수를 갖는 계층이 모델의 state_dict에 항목을 가지게 된다.

일단 모델의 가중치를 저장하려면 3가지 함수를 알아야 한다

1. torch.save
    - 객체를 디스크에 저장합니다. pickle 모듈을 이용하여 객체를 직렬화 하며, 이 함수를 사용하여 모든 종류의 모델, Tensor 등을 저장할 수 있습니다.

    - 형태 : ```torch.save(obj, f, pickle_module=pickle, pickle_protocol=DEFAULT_PROTOCOL, _use_new_zipfile_serialization=True)```

        - 매개변수
            - obj : 저장할 객체
            - f : 파일 유사 개체(쓰기 및 플러시를 구현해야 함) 또는 문자열 또는 os.파일 이름을 포함하는 PathLike 개체, 단순하게 저장할 위치 + 파일명이다
            - pickle_module (Any) : 메타데이터 및 객체를 피클링하는 데 사용되는 모듈
            - pickle_protocol (int) : 기본 프로토콜을 재정의하도록 지정할 수 있습니다.
            #
    - 일반적인 PyTorch 규칙은 .pt 파일 확장자를 사용하여 텐서를 저장하는 것입니다.

    - torch.save(object, path) 예제

In [ ]:
PATH = './weights/'

torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

2. torch.load
    - pickle 모듈을 이용하여 객체를 역직렬화하여 메모리에 할당합니다.

    - 형태 : torch.load(f, map_location=None, pickle_module=pickle, *, weights_only=False, **pickle_load_args)

    - torch.load(path) 이때 path: 불러올 위치 + 파일명


3. torch.nn.Module.load_state_dict: 역직렬화된 state_dict를 사용, 모델의 매개변수들을 불러옵니다. state_dict는 간단히 말해 각 체층을 매개변수 Tensor로 매핑한 Python 사전(dict) 객체입니다.

    - 형태 : torch.nn.Module.loadstatedict(dict)

    - state_dict를 이용하여, 모델 객체 내의 매개 변수 값을 초기화 합니다.

    - 예제


In [ ]:
model = torch.load(PATH + 'model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기

model.load_state_dict(checkpoint['model'])

optimizer.load_state_dict(checkpoint['optimizer'])

모델 가중치를 불러오기 위해서는, 먼저 동일한 모델의 인스턴스(instance)를 생성한 다음에 load_state_dict() 메소드를 사용하여 매개변수들을 불러옵니다.

추론(inference)을 하기 전에 model.eval() 메소드를 호출하여 드롭아웃(dropout)과 배치 정규화(batch normalization)를 평가 모드(evaluation mode)로 설정해야 합니다. 

그렇지 않으면 일관성 없는 추론 결과가 생성됩니다.

In [ ]:
model = models.vgg16() # 기본 가중치를 불러오지 않으므로 pretrained=True를 지정하지 않습니다.
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

### 2. 실전! 모델 저장하기
---
> 참고 출처 : https://076923.github.io/posts/Python-pytorch-10/

모델의 `저장법`은 2가지가 있다.
1. 모델 그 자체를 저장한다.
2. 모델의 상태만을 저장한다.

모델의 `저장 시기`는 2가지가 있다.
1. 모델의 학습이 완전히 끝나고 나서 저장
2. 일정 에폭마다 학습된 결가를 저장


먼저 모델의 저장법부터 알아보자

1. 모델 그 자체를 저장한다
    - torch.save(model, '경로') 모델의 정보를 경로에 저장한다. 모델 파일 확장자는 주로 *.pt나 *.pth 의 확장자를 사용하여 저장함

    - 모델 불러오기 
        - ```model = torch.load("경로", map_location=device)```    
        - 단, 이때 모델을 불러오는 경우에도 동일한 형태의 클래스가 선언되어 있어야 한다. 이때 선언만 되어 있으면 된다. 정의는 pass로 해도 클래스명(상속클래스) 만 되어 있어도 된다
        #

2. 모델의 상태만을 저장한다
    - ```torch.save(model.state_dict(), "./model_state_dict.pt") ```
    - 모델 상태(torch.state_dict)는 모델에서 학습이 가능한 매개변수를 순서가 있는 사전(OrderedDict) 형식으로 반환된다.
    - 순서가 있는 사전에는 레이어 변수의 가중치나 편향 등등 추론에 필요한 데이터가 저장되어 있다.
    #

    - 모델 불러오기
        - ```model_state_dict = torch.load("./model_state_dict.pt", map_location=device)``` 를 먼저 선언해서 데이터를 끌어오고
        - ``` model.load_state_dict(torch.load(save_path)) ```
        - model.load_state_dict(model_state_dict) 모델에 저장한다. 
        - 모델 상태만 저장했기 때문에 순서가 있는 사전에는 그저 가중치나 편향만이 존재한다. 즉 모델이 선언과 정의가 완전히 되어 있어야 한다. 모델 구조가 완전히 있어야 한다는 뜻
        #

모델의 저장 시기에 대해 알아보자
1. 모델의 학습이 완전히 끝나고 나서 저장
    - 그냥 save() 함수를 모델의 학습이 끝나고 나서 사용하면 그만이다
    #

2. 일정 에폭마다 학습된 결과를 저장
    - 보통 층이 깊지 않은 신경망의 경우 이러지 않아도 되는데, 층이 깊은 신경망의 경우 모델이 학습하는데 걸리는 시간이 어마어마하게 늘어난다. 그러다가 저장도 안하고 중간에 날려먹으면 그야말로 대참사
    - 따라서 이런 경우 각 에폭마다 학습된 결과를 저장하게 하면 된다.
    - 이렇게 저장할 경우 체크포인트 저장이라고 한다.

### 3. 체크포인트 저장/불러오기
---

체크포인트(Checkpoint)도 모델 저장 함수(torch.save)를 활용해 여러 상태를 저장한다. 

단, 다양한 정보를 저장하기 위해 사전(Dictionary) 형식으로 값을 할당한다.

학습을 이어서 진행하기 위한 목적이므로, 에폭(Epoch), 모델 상태(model.state_dict), 최적화 상태(optimizer.state_dict) 등은 필수로 포함되어야 한다..

정수형, 실수형, 문자열 등도 함께 저장할 수 있으므로, 부수적인 정보도 함께 포함시킬 수 있다.

1. 저장하는 방법

``` py
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)
```

2. 불러오는 방법

``` py
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH) # 체크포인트를 가져온 뒤 하나씩 할당하기
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - 또는 -
model.train()
```